# Obtain data of AGN from SIMBAD and filtering the objects with data of Ne II, Ne III and Ne V in infrared from NED 

In this notebook we develop the code to Obtain the data of AGNs type Sy1 (Seyfert 1), Sy2 (Seyfert 2) and SyG (Seyfert Galaxies) from **[SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** (SMB) and then search each object from the SMB sample in **[NED](https://ned.ipac.caltech.edu/)** to obtain the photometric data and filter all the objects from the sample that contain data of the emission lines of Ne II, Ne III and Ne V in infrared.

The part of the code to obtain the data from SIMBAD is based in Andres Ramos's work to obtain Seyfert Samples (https://github.com/aframosp/AGNView)

In [ ]:
# If this notebook will be run in Google Colab, first at all the next code must be run. If the notebook will be run in Jupyter
#then "comment" or omit this part 

import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    !pip install astroquery

In [1]:
from datetime import date
import time

import pyvo
import numpy as np
from astroquery.ned import Ned

import astropy.units as u
from astropy.table import hstack
from astropy.coordinates import SkyCoord

Using the package **[PyVO](https://pyvo.readthedocs.io/en/latest/#)** we can access to remote data and services like the **TAP Service of SIMBAD [SIMBAD](http://simbad.u-strasbg.fr/simbad/sim-tap)** to find and retrieve the astronomical data that we need, in this case: Seyfert Galaxies. To achive that, we use a script with **[ADQL](https://www.ivoa.net/documents/latest/ADQL.html) (Astronomical Data Query Language)**, which is based on SQL.

On the script we used the next **Field Names** for VOTable ouput:
- ra: Right ascension
- dec: Declination
- coo_bibcode: Bibliographical reference for coordinates
- otype_txt: Object type
- rvz_bibcode: Bibliographical reference of radial velocity and redshift
- rvz_radvel: Stored value. Either a radial velocity, or a redshift, depending on the rvz_type field 
- rvz_redshift
- rvz_type: Stored type of velocity: 'v'=radial velocity, 'z'=redshift
- nbref: bibliographical reference

~~~~sql
SELECT main_id
	,ra
	,dec
	,coo_bibcode
	,otype_txt
	,rvz_bibcode
	,rvz_radvel
	,rvz_redshift
	,rvz_type
	,nbref
	,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
	OR basic.otype = 'Sy2'
	OR basic.otype = 'SyG'
~~~~

In [2]:
service = pyvo.dal.TAPService("http://simbad.u-strasbg.fr:80/simbad/sim-tap")

result = service.search("""
SELECT main_id
    ,ra
    ,dec
    ,coo_bibcode
    ,otype_txt
    ,rvz_bibcode
    ,rvz_radvel
    ,rvz_redshift
    ,rvz_type
    ,nbref
    ,alltypes.otypes
FROM basic
JOIN alltypes ON oid = oidref
WHERE basic.otype = 'Sy1'
    OR basic.otype = 'Sy2'
    OR basic.otype = 'SyG'
""")

We save the date in a variable for use later in the file names of the results

In [17]:
today = date.today()
today

datetime.date(2021, 10, 7)

We extract the object name (or in this case the main id) from the sample result.

In [4]:
simbad_sample_Sy = result.to_table()
obj_name_SMB=simbad_sample_Sy["main_id"]
simbad_sample_Sy

main_id,ra,dec,coo_bibcode,otype_txt,rvz_bibcode,rvz_radvel,rvz_redshift,rvz_type,nbref,otypes
,deg,deg,,,,km / s,,,,
object,float64,float64,object,object,object,float64,float64,str1,int32,object
LEDA 165630,212.38023750000002,5.192030555555555,2009ApJS..182..543A,Sy2,2011A&A...528A.124C,69068.1659042559,0.2644,z,68,IR |QSO|Sy2|G |G
2MASS J10034653+1222009,150.9439585861228,12.3669990150726,2018yCat.1345....0G,Sy1,2012ApJS..203...21A,71464.57474536476,0.27514,z,17,IR |QSO|AGN|X |BiC|QSO|Sy1|Sy1|AGN
2XMMi J032518.4-060739,51.326875,-6.1273888888888886,2005AJ....129.1755A,Sy1,2009ApJS..182..543A,126754.92483174153,0.57005,z,13,QSO|X |Sy1|AGN
USNO-A2.0 0825-00787515,52.0941222224393,-6.7266691804094,2018yCat.1345....0G,Sy1,2009ApJS..182..543A,93552.23834660952,0.38102,z,15,QSO|Sy1|AGN|* |UV
2SLAQ J033852.90+001904.7,54.7204660377791,0.3179910253349,2018yCat.1345....0G,Sy1,2009ApJS..182..543A,108224.28742924411,0.45941,z,25,X |QSO|QSO|AGN|* |Sy1|AGN
2SLAQ J034007.62-004632.5,55.0317750936304,-0.7757204237907,2018yCat.1345....0G,Sy1,2009ApJS..182..543A,138971.58950556486,0.65175,z,19,QSO|QSO|Sy1|* |QSO|AGN
2XMM J032328.4-371009,50.86844,-37.16924,2012ApJ...756...27L,Sy1,,--,--,,1,X |Sy1
2MASS J13241017+6410056,201.042164402469,64.1681640803473,2018yCat.1345....0G,Sy1,2012ApJS..203...21A,89131.01929523506,0.35875,z,15,IR |QSO|AGN|X |QSO|* |Sy1|AGN|X


The sample results is saved in a .csv file:

In [18]:
simbad_sample_Sy.write(today.strftime('%b_%d_%Y')+'_Sy_Samples'+'.csv',
                    format='ascii.csv',overwrite=True)

Using **SkyCoord** from astropy we extract the coordinates information for each object from the SIMBAD samples. This is because for some objects when we search the information in [NED](https://ned.ipac.caltech.edu/) using their SIMBAD main id  it doesn't reconigze it, so we will use coordinates instead to get the NED object name

In [5]:
cat_smb_sample = SkyCoord(ra=simbad_sample_Sy['ra'], dec=simbad_sample_Sy['dec'])

The wavelenght for NeII, NeIII and NeV in the infrared spectrum are 12.8 microns, 15.6 microns and 14.3 microns respectively. This values are equivalents to the following frequencies which ara related to different labels of "Observed Passband" in NED database as we ilustrate in the next list:

* Frequency: 1.92e+13 Hz: [NeIII] 15.6 microns
* Frecuency: 1.93e+13 Hz: [Ne III] 15.56 (IRS), [NeIII] 15.6 Spitzer, [Ne III] (Spitzer), [Ne III] 15.5 (IRS)
* Frecuency: 2.09e+13 Hz: [Ne V] 14.32 (IRS), [Ne V] 14.32 Spitzer, [Ne V] 14.32, [Ne V] 14.3 Spitzer, [Ne V] 14.3 microns
* Frequency: 2.10e+13 Hz: [Ne V] 14.3 (IRS)
* Frequency: 2.34e+13 Hz: [Ne II] 12.81 (IRS), [Ne II] 12.8 Spitzer, [Ne II] 12.8 (IRS), [Ne II] 12.81, NEII (VLT), 12.81 microns (VLT) 

In the variable **Ne_IR_Freq** we define a list with the those frequency values

In [6]:
Ne_IR_Freq = [19200000000000.0,19300000000000.0,20900000000000.0,21000000000000.0,23400000000000.0]

Using the coordinates extracted with SkyCoord and using [Astroquery](https://astroquery.readthedocs.io/en/latest/index.html) package we will get the information from NED for each object in SIMBAD sample. Then we took the **Object Name** to get the **Photometry table** of this objects. The filter for the objects with Ne Line Emissions in Infrared will be looking in the column **"Frequency"** of the table the values of the list defined in Ne_IR_Freq.

In [7]:
def nedtablephot(obj_name):
    try:
        #We get the photometry table from NED for each sample
        phot_table = Ned.get_table(obj_name, table='photometry')
        error = 0
    except:
        error = 1
        phot_table=[]
    return phot_table, error
        

In [8]:
def nedqueryregion(coords):
    try: 
        result_table = Ned.query_region(coords,radius=0.0005*u.deg) #For obtaining the data from NED for each sample
        obj_name_NED = result_table["Object Name"]
        error = 0
    except:
        error = 1
        obj_name_NED=[]
    return obj_name_NED, error

In [10]:
Ne_IR_obj = [] #List of objects with Ne-IR data

outfile = open('Obj_with_Ne-IR.txt','w') #We create and open a file to write the list of objects names with Ne_IR data
headerstring = 'Objects with Ne II, Ne III and Ne V data:\n' #Creating the header
outfile.write(headerstring) #Writing the header

start = time.time()
for i in range(6000):
    
    k=i #Flag to print (and save) only the photometric data of objects with Ne-IR data
    phot_table1,error1 = nedtablephot(obj_name_SMB[i])
    len_table = len(phot_table1) #Lenght of the photometry data
    
    if error1 == 0:

        for j in range(len_table):
            Freq=phot_table1["Frequency"][j] #We get the values of the Frequencies observed

            if (Freq==Ne_IR_Freq[0] or Freq==Ne_IR_Freq[1] or Freq==Ne_IR_Freq[2] or Freq==Ne_IR_Freq[3] or 
                Freq==Ne_IR_Freq[4]): 
                
                #If k is equal to the current sample indice, save and print onlyone time the object name 
                #and print the table title
                if k==i: 
                    #print('\n')
                    print(f"Index No.:{i}, Name(SIMBAD): {obj_name_SMB[i]}")
                    #Writing the object names
                    outstring = f'\n{obj_name_SMB[i]}'
                    outfile.write(outstring) 
                    Ne_IR_obj.append(obj_name_SMB[i]) #Save the object name in the list
                    
                k+=1
    elif error1 == 1:
        obj_name_NED, error2 = nedqueryregion(cat_smb_sample[i])
        
        if error2 == 0:
            phot_table2,error3 = nedtablephot(obj_name_NED)
            len_table1 = len(phot_table2) #Lenght of the photometry data
            if error3 == 0:
                for j in range(len_table1):
                    Freq=phot_table2["Frequency"][j] #We get the values of the Frequencies observed

                    if (Freq==Ne_IR_Freq[0] or Freq==Ne_IR_Freq[1] or Freq==Ne_IR_Freq[2] or Freq==Ne_IR_Freq[3] or 
                        Freq==Ne_IR_Freq[4]): 
                
                #If k is equal to the current sample indice, save and print onlyone time the object name 
                #and print the table title
                        if k==i: 
                            print(f"Index No.:{i}, Name(NED): {obj_name_NED[0]}")
                            #Writing the object names
                            outstring = f'\n{obj_name_NED[0]}'
                            outfile.write(outstring) 
                            Ne_IR_obj.append(obj_name_NED[0]) #Save the object name in the list
                    
                        k+=1
            
end = time.time()
#we close the file with objects names
outfile.close()

print('\nTotal objects with Ne II or Ne III or Ne V data found: ',len(Ne_IR_obj))
print('Execution Time(seg): ' + str(end - start))
print('Execution Time(min): ' + str((end - start)/60))
print('Execution Time(hrs): ' + str((end - start)/3600))

Index No.:0, Name(SIMBAD): LEDA  165630
Index No.:23, Name(SIMBAD): ESO 362-18
Index No.:100, Name(NED): 3C 334
Index No.:182, Name(SIMBAD): ESO 323-77
Index No.:342, Name(SIMBAD): NGC  7479
Index No.:400, Name(SIMBAD): M  81
Index No.:408, Name(SIMBAD): NGC  1241
Index No.:409, Name(SIMBAD): NGC  3516
Index No.:429, Name(SIMBAD): IRAS 15206+3342
Index No.:450, Name(SIMBAD): LEDA 1487448
Index No.:611, Name(SIMBAD): Mrk  937
Index No.:624, Name(SIMBAD): ESO 297-18
Index No.:644, Name(SIMBAD): IC  450
Index No.:790, Name(SIMBAD): Mrk  622
Index No.:858, Name(NED): WISEA J035734.06-053719.6
Index No.:1051, Name(SIMBAD): 2MASS J21035875-0728025
Index No.:1111, Name(SIMBAD): IRAS 15480-0344
Index No.:1411, Name(SIMBAD): Mrk  471
Index No.:1585, Name(SIMBAD): 2MASS J00105486+0014510
Index No.:1588, Name(NED): WISEA J005439.78+155447.2
Index No.:1869, Name(SIMBAD): LEDA  166692
Index No.:2266, Name(SIMBAD): NGC  4992
Index No.:2624, Name(SIMBAD): 3C 295
Index No.:2852, Name(SIMBAD): LEDA    

In [11]:
Ne_IR_obj

['LEDA  165630',
 'ESO 362-18',
 '3C 334',
 'ESO 323-77',
 'NGC  7479',
 'M  81',
 'NGC  1241',
 'NGC  3516',
 'IRAS 15206+3342',
 'LEDA 1487448',
 'Mrk  937',
 'ESO 297-18',
 'IC  450',
 'Mrk  622',
 'WISEA J035734.06-053719.6',
 '2MASS J21035875-0728025',
 'IRAS 15480-0344',
 'Mrk  471',
 '2MASS J00105486+0014510',
 'WISEA J005439.78+155447.2',
 'LEDA  166692',
 'NGC  4992',
 '3C 295',
 'LEDA    3237',
 'Mrk  478',
 'NGC  1125',
 'Mrk  279',
 'Mrk 1502',
 'PB  3894',
 'ICRF J054236.1+495107',
 '3C 098',
 'NGC  2992',
 'NGC  2110',
 'UGC  3351',
 'NGC   788',
 'NGC  4235',
 'NGC  6860',
 'Mrk  520',
 'MCG+04-48-002',
 'ICRF J145907.5+714019',
 '2E  2503',
 'MCG+00-29-023',
 'Z 218-7',
 'Mrk  463E',
 'LEDA   27453',
 'IRAS 07598+6508',
 'NGC  4151',
 'NGC  1365',
 '2XMM J101756.8+390526',
 '3C  84',
 'ESO 141-55',
 'MCG+10-17-021',
 'Mrk  509',
 'NGC  7591',
 'Mrk  493',
 'Mrk 1239',
 'Mrk  231',
 'NGC  3079',
 'NGC  4355',
 'IRAS 13342+3932',
 'WISEA J165939.77+183436.7',
 'Mrk  876',

Las celdas a continuación son para pruebas:

In [14]:
#Celda para pruebas
ind=400

result_t = Ned.query_region(cat_smb_sample[ind],radius=0.0005 * u.deg) #For obtaining the data from NED for each sample 

o_name=result_t["Object Name"]
try:
  phot_t = Ned.get_table(o_name, table='photometry')
  #print(phot_t['Frequency'])
  len_t=len(phot_t)
  for a in range(len_t):
    Passb=phot_t['Frequency'][a] #We get the name for each Observed Passband
    if (Passb==Ne_IR_Freq[0] or Passb==Ne_IR_Freq[1] or Passb==Ne_IR_Freq[2] or Passb==Ne_IR_Freq[3] or Passb==Ne_IR_Freq[4]):
      print(o_name)
except:
  pass

In [15]:
result_t

No.,Object Name,RA,DEC,Type,Velocity,Redshift,Redshift Flag,Magnitude and Filter,Separation,References,Notes,Photometry Points,Positions,Redshift Points,Diameter Points,Associations
,,degrees,degrees,,km / s,,,,arcmin,,,,,,,
int32,str30,float64,float64,object,float64,float64,object,object,float64,int32,int32,int32,int32,int32,int32,int32
1,NGC 3031:[MKH2012] CLU161,148.888,69.065,UvES,--,--,,,0.018,2,0,0,2,0,0,0
2,CXO J095533.1+690355,148.88805,69.06531,XrayS,--,--,,,0.004,2,0,0,2,0,0,1
3,CXO J095533.14+690355.1,148.88812,69.06532,XrayS,--,--,,,0.003,4,0,1,2,0,0,0
4,2MASS J09553315+6903550,148.88816,69.06529,IrS,--,--,,,0.001,0,0,6,1,0,0,0
5,SSTSL2 J095533.16+690355.6,148.8882,69.06546,IrS,--,--,,,0.01,0,0,8,1,0,0,0
6,MESSIER 081,148.88822,69.06529,G,-39.0,-0.00013,,7.89,0.0,2547,48,283,88,45,8,1
7,WISEA J095533.22+690354.7,148.88845,69.06522,IrS,--,--,,,0.007,0,0,12,1,0,0,0
8,CXO J095533.2+690356,148.88857,69.06566,XrayS,--,--,,,0.023,2,0,0,2,0,0,0


In [16]:
cat_smb_sample[ind]

<SkyCoord (ICRS): (ra, dec) in deg
    (148.88822109, 69.0652947)>

In [9]:
%load_ext watermark
%watermark -a "Jonhatan Bernal" -d -v -m
print('Specific Python packages')
%watermark -iv -w --packages astropy

Author: Jonhatan Bernal

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.27.0

Compiler    : MSC v.1928 64 bit (AMD64)
OS          : Windows
Release     : 10
Machine     : AMD64
Processor   : AMD64 Family 21 Model 2 Stepping 0, AuthenticAMD
CPU cores   : 8
Architecture: 64bit

Specific Python packages
astropy: 4.3.1

pyvo   : 1.1
numpy  : 1.20.3
astropy: 4.3.1

Watermark: 2.2.0

